In [1]:
import sys
sys.path.insert(0, '/home/leandro/math/sage/sage_tvars')
from tvars import *
import random
PS=ProjectiveSpace(1,QQ)
BR = QQ;

In [2]:
def p_divisor_gen(p, bound):
    # generate a p-divisor of type (1,p,q) and dimension 3 
    #bound = 5
    n=2
    q = p+1;
    v0= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v_inf= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    while v0[0] % p == 0 or v0[1] % p == 0:
        v0= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v0 = (1/p)*v0
    while (v_inf[0] % q == 0) or (v_inf[1] % q == 0):
         v_inf= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v_inf = (1/q)*v_inf
    v1= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v2 = vector([0,0])
    #v2= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    sigma = Cone([v0 + v1 + v_inf, v0 + v2 + v_inf]).rays()
    # the p-divisor
    dict = {PS(0,1) : Polyhedron(vertices=[v0],rays=sigma, base_ring=BR),
           PS(1,0) : Polyhedron(vertices=[v_inf],rays=sigma, base_ring=BR),
           PS(1,1) : Polyhedron(vertices =[v1,v2],rays=sigma, base_ring=BR)
           }
    D = PolyhedralDivisor(dict);
    if not D.is_proper():
        #print("not proper")
        raise NotImplementedError(f"The variety is not Q-Gorenstein.")
    if not D.is_Q_Gorenstein():
        #print("Not Q-G")
        raise NotImplementedError(f"The variety is not Q-Gorenstein.")
    return D

In [3]:
def p_divisor_gen3(p, bound):
    # generate a p-divisor of type (1,p,q) and dimension 4
    #bound = 5
    n=3
    q = p+1;
    v0= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v_inf= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    while v0[0] % p == 0 or v0[1] % p == 0 or v0[2] % p == 0:
        v0= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v0 = (1/p)*v0
    while (v_inf[0] % q == 0) or (v_inf[1] % q == 0) or (v_inf[2] % q == 0):
         v_inf= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v_inf = (1/q)*v_inf
    v1= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v2 = vector([0,0,0])
    v3= vector([random.choice(range(-bound, bound)), random.choice(range(-bound, bound)), random.choice(range(-bound, bound))])
    v4 = vector([0,0,0])
    sigma = Cone([v0 + v1 + v3 + v_inf, v0 + v1 + v4 + v_inf, v0 + v2 + v3 + v_inf, v0 + v2 + v4 + v_inf]).rays()
    # the p-divisor
    dict = {PS(0,1) : Polyhedron(vertices=[v0],rays=sigma, base_ring=BR),
           PS(1,0) : Polyhedron(vertices=[v_inf],rays=sigma, base_ring=BR),
           PS(1,1) : Polyhedron(vertices =[v1,v2],rays=sigma, base_ring=BR),
           PS(1,2) : Polyhedron(vertices =[v3,v4],rays=sigma, base_ring=BR)
           }
    D = PolyhedralDivisor(dict);
    if not D.is_proper():
        print("not proper")
        raise NotImplementedError(f"The variety is not Q-Gorenstein.")
    if not D.is_Q_Gorenstein():
        print("Not Q-G")
        raise NotImplementedError(f"The variety is not Q-Gorenstein.")
    return D

In [15]:
%%script false --no-raise-error
n=2
v0= vector([(1/p), 0])
v_inf= vector([(1/q)*(-1), 0])
v1= vector([1,0])
v2 = vector([1,0])
sigma = Cone([[1,1],[1,0]]).rays()
dict = {PS(0,1) : Polyhedron(vertices=[v0],rays=sigma, base_ring=BR),
       PS(1,0) : Polyhedron(vertices=[v_inf],rays=sigma, base_ring=BR),
       PS(1,1) : Polyhedron(vertices =[v1],rays=sigma, base_ring=BR),
       PS(2,1) : Polyhedron(vertices =[v2],rays=sigma, base_ring=BR)
       }
D3 = PolyhedralDivisor(dict);
if not D3.is_proper():
    raise NotImplementedError(f"The variety is not Q-Gorenstein.")
if not D3.is_Q_Gorenstein():
    raise NotImplementedError(f"The variety is not Q-Gorenstein.")
D3

In [18]:
def divisors(n, k):
    #dim(X) = n+1
    divs = []
    for i in range(k):
        p = 3
        #p = random.choice(range(15,30))
        q = p+1
        #q = random.choice(range(p+1, 2*p))
        try:
            if n == 3:
                D = p_divisor_gen3(p,25)
            else:
                D = p_divisor_gen(p, 25)
        except IndexError as ie:
            print("index trouble")
            continue
        # the variety should always be Q-Gorenstein but apparently sometimes isn't 
        except NotImplementedError as e:
            print("Not Q-G.")
            continue
        if D.multiplicity_of_slice(PS(0,1)) != p:
            continue
        if D.multiplicity_of_slice(PS(1,0)) != q:
            continue
        divs.append(D)
    return divs

In [19]:
def multiplicities(D, y1,y2):
    n = D.tail.dim() + 1
    C1 = D.degeneration(y1)
    w = vector(QQ, sum(C1.rays()[0:n]))
    w_n = (-1/w[n-1])*w
    mu_w = lcm(w_i.denominator() for w_i in w_n if w_i != 0)
    l1 = D.vert_mld(y2)
    for key, value in l1.items():
        if mu_w in value:
            return [True, y1]
    C2 = D.degeneration(y2)
    x = vector(QQ, sum(C2.rays()[0:n]))
    x_n = (-1/x[n-1])*x
    mu_x = lcm(x_i.denominator() for x_i in x_n if x_i != 0)
    l2 = D.vert_mld(y1)
    for key, value in l2.items():
        if mu_x in value:
            return [True,y2]
    return False

In [20]:
def ray_sum(D, y1, y2, ray_list= None):
    # TODO: look for rays that make sense to sum up
    n = D.tail.dim() + 1
    C1 = D.degeneration(y2)
    if ray_list == None:
        ray_list = [int(i) for i in range(n)]
        #print(ray_list)
    w = vector(QQ, sum(C1.rays()[i] for i in ray_list))
    if w[n-1] >= 0:
        return w
    else:
        w_n = (-1/w[n-1])*w
        #w_n = w_n*w[0:n-1]
        mu_w = lcm(w_i.denominator() for w_i in w_n if w_i != 0)
        return [w_n, D.coefficient(y1).relative_interior_contains(w_n)]

In [21]:
def log_disc_of_ray_sum(D, y1, y2, ray_list):
    n = D.tail.dim() + 1
    C1 = D.degeneration(y2)
    #w = vector(QQ, sum(C1.rays()[0:n-1]) + vector(C1.rays()[n]))
    #w = vector(QQ, sum(C1.rays()[0:n]))
    w = vector(QQ, sum(C1.rays()[i] for i in ray_list))
    w_n = (-1/w[n-1])*w[0:n-1]
    mu_w = lcm(w_i.denominator() for w_i in w_n if w_i != 0)
    if D.coefficient(y1).relative_interior_contains(w_n):
        return D.vert_mld_of_vertex(y1, w_n)
    else:
        return -10
    #return D.coefficient(y1).relative_interior_contains(w_n)
    #if D.coefficient(y1).relative_interior_contains(w_n - vector(D.coefficient(PS(1,1)).vertices()[0])- vector(D.coefficient(PS(2,1)).vertices()[0])):
    #if D.coefficient(y1).relative_interior_contains(w_n):
        #return D.vert_mld_of_vertex(y1,w_n)
    #else:
        #return False

In [22]:
def mld_normalized_volume(D):
    n = dim(D.tail) + 1
    mld = D.mld(PS(1,2))
    try:
        # don't quite know why this goes wrong sometimes but we catch it to avoid trouble
        nvol = D.nvol_of_val(D.minimize_nvol())
    except ValueError as ve:
        return 0
    return mld* n^(n-1) >= nvol

In [23]:
divs2 = divisors(3,35)
len(divs2)

34

In [27]:
for D in divs2:
    print(D.vert_mld_of_vertex(PS(0,1),[ray_sum(D, PS(1,0),PS(0,1))[0]]))

['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative interior']
['vector not contained in relative

In [153]:
l = [int(i) for i in range(3)]
res0 = [D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0),l)).bit_count() for D in divs2]
[i for i in res if i == 0]

[]

In [154]:
l = [int(i) for i in range(3)]
res_inf = [D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D,PS(1,0), PS(0,1),l)).bit_count() for D in divs2]
#[i for i in res_inf if i == 0]
res_inf

[0]

In [148]:
[divs2[0].coefficient(p).vertices() for p in divs2[0].supp()] + [divs2[0].tail.rays()]

[(A vertex at (3/16, -3/4),),
 (A vertex at (-16/17, -12/17),),
 (A vertex at (-15, 7), A vertex at (0, 0)),
 N(-4285, 1508),
 N( -205, -396)
 in 2-d lattice N]

5

In [19]:
divs = divisors(3,5)
len(divs)

0

In [105]:
# generate a bunch of 4-dimensional examples and test things, e.g. whether there is a hope of 
# ending up with an element of int(Delta_0) by summing up some rays

l0 = [int(d) for d in range(1,5)] #skip 0
l1 = [int(d) for d in range(1)] + [int(d) for d in range(2,5)] #skip 1
l2 = [int(d) for d in range(2)] + [int(d) for d in range(3,5)] #skip 2
l3 = [int(d) for d in range(3)] + [int(d) for d in range(4,5)] #skip 3
l4 = [int(d) for d in range(4)] #skip 4

R = []
R.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l0)).bit_count() for D in divs[:]])
R.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l1)).bit_count() for D in divs[:]])
R.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l2)).bit_count() for D in divs[:]])
R.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l3)).bit_count() for D in divs[:]])
R.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l4)).bit_count() for D in divs[:]])

# we see that for some of these p_divisors, there is in these exampes always configuration 
# of the form "sum of 4 rays" that ends up inside int(Delta_0)
[max(R[0][i],R[1][i], R[2][i],R[3][i], R[4][i]) for i in range(len(R[0]))]

[1, 1, 1, 1, 1]

In [113]:
[R[i] for i in range((len(R)))]

[[0, 0, 1, 1, 0],
 [0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1]]

In [116]:
log_disc_of_ray_sum(divs[0], PS(0,1), PS(1,0), l2)

4

In [117]:
log_disc_of_ray_sum(divs[1], PS(0,1), PS(1,0), l1)

4

In [118]:
log_disc_of_ray_sum(divs[2], PS(0,1), PS(1,0), l0)

4

In [119]:
log_disc_of_ray_sum(divs[3], PS(0,1), PS(1,0), l0)

4

In [120]:
log_disc_of_ray_sum(divs[4], PS(0,1), PS(1,0), l4)

4

In [106]:
[D.degeneration(PS(1,0)).rays() for D in divs]

[N( 68,  50, -71, -3),
 N(-19, -25, -38, -3),
 N( 50,  32, -86, -3),
 N( -1,  -7, -23, -3),
 N(  1,   5,  18,  4)
 in 4-d lattice N,
 N(-9,  -3, -21,  4),
 N(95, -79,  20, -3),
 N( 8,  -1,   8, -3),
 N(26, -31, -31, -3),
 N(77, -49,  59, -3)
 in 4-d lattice N,
 N( 95,  61, -94, -3),
 N(-25,  15,  17,  4),
 N(-13, -23,  -7, -3),
 N( 38,  34, -67, -3),
 N( 44,   4, -34, -3)
 in 4-d lattice N,
 N( 79, 109, -82, -3),
 N(-17,  19,  -7, -3),
 N(-13, -18,  11,  4),
 N( 25,  46, -73, -3),
 N( 37,  82, -16, -3)
 in 4-d lattice N,
 N( 62, 62,  4, -3),
 N(-16, 23, 82, -3),
 N(-15, 19, -6,  4),
 N( 17,  2, 13, -3),
 N( 29, 83, 73, -3)
 in 4-d lattice N]

In [40]:
divs[0].coefficient(PS(1,1)).vertices()

(A vertex at (-8, -21, -8), A vertex at (0, 0, 0))

In [ ]:
R0 = []
R0.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l0)).bit_count() for D in divs[:]])
R0.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l0) - vector(D.coefficient(PS(1,2)).vertices()[1])).bit_count() for D in divs[:]])
R0.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l0) - vector(D.coefficient(PS(1,1)).vertices()[0])).bit_count() for D in divs[:]])
R0.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l0) - vector(D.coefficient(PS(1,2)).vertices()[1]) - vector(D.coefficient(PS(1,1)).vertices()[0])).bit_count() for D in divs[:]])
R1 = []
R1.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l1)).bit_count() for D in divs[:]])
R1.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l1) - vector(D.coefficient(PS(1,2)).vertices()[1])).bit_count() for D in divs[:]])
R1.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l1) - vector(D.coefficient(PS(1,1)).vertices()[0])).bit_count() for D in divs[:]])
R1.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l1) - vector(D.coefficient(PS(1,2)).vertices()[1]) - vector(D.coefficient(PS(1,1)).vertices()[0])).bit_count() for D in divs[:]])
R2 = []
R2.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l2)).bit_count() for D in divs[:]])
R2.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l2) - vector(D.coefficient(PS(1,2)).vertices()[1])).bit_count() for D in divs[:]])
R2.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l2) - vector(D.coefficient(PS(1,1)).vertices()[0])).bit_count() for D in divs[:]])
R2.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l2) - vector(D.coefficient(PS(1,2)).vertices()[1]) - vector(D.coefficient(PS(1,1)).vertices()[0])).bit_count() for D in divs[:]])
R3 = []
R3.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l3)).bit_count() for D in divs[:]])
R3.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l3) - vector(D.coefficient(PS(1,2)).vertices()[1])).bit_count() for D in divs[:]])
R3.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l3) - vector(D.coefficient(PS(1,1)).vertices()[0])).bit_count() for D in divs[:]])
R3.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l3) - vector(D.coefficient(PS(1,2)).vertices()[1]) - vector(D.coefficient(PS(1,1)).vertices()[0])).bit_count() for D in divs[:]])
R4 = []
R4.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l4)).bit_count() for D in divs[:]])
R4.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l4) - vector(D.coefficient(PS(1,2)).vertices()[1])).bit_count() for D in divs[:]])
R4.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l4) - vector(D.coefficient(PS(1,1)).vertices()[0])).bit_count() for D in divs[:]])
R4.append([D.coefficient(PS(0,1)).relative_interior_contains(ray_sum(D, PS(0,1),PS(1,0), l4) - vector(D.coefficient(PS(1,2)).vertices()[1]) - vector(D.coefficient(PS(1,1)).vertices()[0])).bit_count() for D in divs[:]])

In [14]:
[multiplicities(D,PS(0,1),PS(1,0)) for D in divs2]

[[True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0 : 1)],
 [True, (0

In [20]:
[mld_normalized_volume(D) for D in divs2]

capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.


[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 0,
 True,
 True,
 True]

In [18]:
556*(501/509) > 557

False